In [2]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [3]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2879,5.7,0.37,0.30,1.10,0.029,24.0,88.0,0.98883,3.18,0.39,11.700000,6
2525,7.3,0.17,0.24,8.10,0.121,32.0,162.0,0.99508,3.17,0.38,10.400000,8
1508,6.8,0.21,0.49,14.50,0.060,50.0,170.0,0.99910,3.55,0.44,9.800000,6
1044,7.2,0.21,0.37,1.60,0.049,23.0,94.0,0.99240,3.16,0.48,10.900000,7
804,7.3,0.32,0.34,6.60,0.032,24.0,112.0,0.99505,3.22,0.46,9.800000,6
2964,5.8,0.19,0.24,1.30,0.044,38.0,128.0,0.99362,3.77,0.60,10.600000,5
510,6.7,0.27,0.12,1.30,0.041,62.0,138.0,0.99210,3.21,0.42,10.000000,6
4401,7.1,0.38,0.29,13.60,0.041,30.0,137.0,0.99461,3.02,0.96,12.100000,6
2960,7.7,0.11,0.34,14.05,0.040,41.0,114.0,0.99634,3.07,0.59,11.000000,7
4880,6.6,0.34,0.40,8.10,0.046,68.0,170.0,0.99494,3.15,0.50,9.533333,6


In [4]:
remote_server_uri = "http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env


In [5]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5000'

In [6]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2024/03/15 12:50:50 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/priyanka/MLFlow/mlruns/199997558282582496', creation_time=1710487250263, experiment_id='199997558282582496', last_update_time=1710487250263, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [7]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [8]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160691
  R2: 0.12678721972772689
Save to: /home/priyanka/MLFlow/mlruns/199997558282582496/ff3ca7feda3445878a1ab99b444b28ba/artifacts


In [9]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062342
  MAE: 0.6155290394093895
  R2: 0.2022463182289208
Save to: /home/priyanka/MLFlow/mlruns/199997558282582496/2631745abda64f239a9a2c851be3956c/artifacts


In [10]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: /home/priyanka/MLFlow/mlruns/199997558282582496/58658187ab5f43bb9b18b0fcb0095d60/artifacts
